# LLaDA Inference Profiling on HumanEval

This notebook runs inference on the LLaDA model using the HumanEval dataset and collects statistics for profiling.

## Load Model and Dataset

In [ ]:
import os
import subprocess
import torch
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

os.environ['HF_HOME'] = '/root/LLaDA/hf_models/'
from transformers import AutoTokenizer, AutoModel

from generate import generate

# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load Model and Tokenizer
model_id = 'GSAI-ML/LLaDA-8B-Instruct'
cache_path = '/root/LLaDA/hf_models/hub'

print(f"Loading model: {model_id}")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_path, local_files_only=True)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.bfloat16, cache_dir=cache_path, local_files_only=True).to(device).eval()

# Ensure padding side is left for generation
if tokenizer.padding_side != 'left':
    tokenizer.padding_side = 'left'

print(model)

In [ ]:
# Load HumanEval Dataset
print("Loading HumanEval dataset...")
dataset_path = '/root/LLaDA/hf_models/datasets/openai_humaneval'
ds = load_dataset(path=dataset_path, split="test")
print(f"Loaded {len(ds)} problems.")

# Display a sample
print("\nSample Problem:")
print(ds[0]['prompt'])


## Run Model Inference and Collect Statistics

In [ ]:
import torch.nn.functional as F

def add_gumbel_noise(logits, temperature):
    '''
    The Gumbel max is a method for sampling categorical distributions.
    According to arXiv:2409.02908, for MDM, low-precision Gumbel Max improves perplexity score but reduces generation quality.
    Thus, we use float64.
    '''
    if temperature == 0:
        return logits
    logits = logits.to(torch.float64)
    noise = torch.rand_like(logits, dtype=torch.float64)
    gumbel_noise = (- torch.log(noise)) ** temperature
    return logits.exp() / gumbel_noise


def get_num_transfer_tokens(mask_index, steps):
    '''
    In the reverse process, the interval [0, 1] is uniformly discretized into steps intervals.
    Furthermore, because LLaDA employs a linear noise schedule (as defined in Eq. (8)),
    the expected number of tokens transitioned at each step should be consistent.

    This function is designed to precompute the number of tokens that need to be transitioned at each step.
    '''
    mask_num = mask_index.sum(dim=1, keepdim=True)

    base = mask_num // steps
    remainder = mask_num % steps

    num_transfer_tokens = torch.zeros(mask_num.size(0), steps, device=mask_index.device, dtype=torch.int64) + base

    for i in range(mask_num.size(0)):
        num_transfer_tokens[i, :remainder[i]] += 1

    return num_transfer_tokens


class ActivationProfiler:
    def __init__(self, model, target_layers=['k_proj', 'v_proj', 'o_proj'], save_dir='../autodl-tmp/profiling_results'):
        self.model = model
        self.target_layers = target_layers
        self.hooks = []
        self.save_dir = save_dir
        self.current_step = 0
        self.buffer = {} # name -> tensor
        os.makedirs(self.save_dir, exist_ok=True)

    def register_hooks(self):
        self.clear()
        for name, module in self.model.named_modules():
            # Check if the module name ends with any of the target layers
            if any(name.endswith(t) for t in self.target_layers):
                # We only want leaf modules, usually
                hook = module.register_forward_hook(self.get_hook(name))
                self.hooks.append(hook)
        print(f"Registered hooks on {len(self.hooks)} layers.")

    def get_hook(self, name):
        def hook(module, input, output):
            # output shape: [batch, seq_len, hidden_dim]
            # Buffer to CPU memory instead of saving immediately to avoid I/O crashes
            self.buffer[name] = output.detach().cpu()
        return hook
    
    def step(self, current_step_index):
        """Sets the current step index for saving."""
        self.current_step = current_step_index

    def save_buffer(self):
        """Saves buffered tensors to disk sequentially."""
        if not self.buffer:
            return
            
        # Create a directory for the current step
        step_dir = os.path.join(self.save_dir, f"step_{self.current_step}")
        os.makedirs(step_dir, exist_ok=True)
        
        for name, tensor in self.buffer.items():
            # Replace dots with underscores for safe filenames
            safe_name = name.replace('.', '_')
            file_path = os.path.join(step_dir, f"{safe_name}.pt")
            torch.save(tensor, file_path)
        
        # Clear buffer to free memory
        self.buffer = {}

    def clear(self):
        for h in self.hooks:
            h.remove()
        self.hooks = []
        self.buffer = {}
        self.current_step = 0
        
    def get_collected_data(self):
        return self.save_dir


def save_token_state(save_dir, step_idx, prompt_mask, mask_index, transfer_index, block_idx, inblock_step):
    """Persist token-role metadata for later visualization."""
    os.makedirs(save_dir, exist_ok=True)
    step_dir = os.path.join(save_dir, f"step_{step_idx}")
    os.makedirs(step_dir, exist_ok=True)
    payload = {
        "prompt_mask": prompt_mask.to(dtype=torch.bool).cpu(),
        "mask_index": mask_index.to(dtype=torch.bool).cpu(),
        "transfer_index": transfer_index.to(dtype=torch.bool).cpu(),
        "block_idx": block_idx,
        "inblock_step": inblock_step,
    }
    torch.save(payload, os.path.join(step_dir, "token_state.pt"))


@ torch.no_grad()
def profiled_generate(model, prompt, attention_mask=None, steps=128, gen_length=128, block_length=128, temperature=0.,
             cfg_scale=0., remasking='low_confidence', mask_id=126336, logits_eos_inf=False, confidence_eos_eot_inf=False, profiler=None):
    '''
    Revised generate function adhering to generate.py logic with profiling.
    '''
    # Initialize x with mask
    x = torch.full((prompt.shape[0], prompt.shape[1] + gen_length), mask_id, dtype=torch.long).to(model.device)
    x[:, :prompt.shape[1]] = prompt.clone()

    if attention_mask is not None:
        attention_mask = torch.cat([attention_mask, torch.ones((prompt.shape[0], gen_length), dtype=attention_mask.dtype, device=model.device)], dim=-1)

    prompt_index = (x != mask_id)

    assert gen_length % block_length == 0
    num_blocks = gen_length // block_length

    assert steps % num_blocks == 0
    steps = steps // num_blocks

    # Profiling data storage
    step_details = []
    
    # Print inference statistics
    print(f'Generation length: {gen_length}')
    print(f'Denoising steps per block: {steps}')
    print(f'Number of blocks: {num_blocks}')

    # Helper for CUDA synchronization
    def synchronize():
        if x.device.type == 'cuda':
            torch.cuda.synchronize()

    synchronize()
    total_start = time.perf_counter()
    for num_block in range(num_blocks):
        block_mask_index = (x[:, prompt.shape[1] + num_block * block_length: prompt.shape[1] + (num_block + 1) * block_length:] == mask_id)
        num_transfer_tokens = get_num_transfer_tokens(block_mask_index, steps)
        
        for i in range(steps):
            global_step = num_block * steps + i
            if profiler:
                # print(f"flag step: global={global_step} block={num_block} in_block={i}")
                profiler.step(global_step)
            
            synchronize()
            step_start = time.perf_counter()
            
            mask_index = (x == mask_id)
            
            # 1. Model Forward Pass
            t_forward_start = time.perf_counter()
            if cfg_scale > 0.:
                un_x = x.clone()
                un_x[prompt_index] = mask_id
                x_ = torch.cat([x, un_x], dim=0)
                if attention_mask is not None:
                    attention_mask_ = torch.cat([attention_mask, attention_mask], dim=0)
                logits = model(x_, attention_mask=attention_mask_).logits
                logits, un_logits = torch.chunk(logits, 2, dim=0)
                logits = un_logits + (cfg_scale + 1) * (logits - un_logits)
            else:
                logits = model(x, attention_mask=attention_mask).logits
            synchronize()
            t_forward_end = time.perf_counter()

            # Save buffered activations for this (global) step
            if profiler:
                profiler.save_buffer()

            if logits_eos_inf:
                logits[:, :, 126081] = -torch.inf

            # 2. Sampling
            t_sample_start = time.perf_counter()
            logits_with_noise = add_gumbel_noise(logits, temperature=temperature)
            x0 = torch.argmax(logits_with_noise, dim=-1) # b, l
            synchronize()
            t_sample_end = time.perf_counter()

            if confidence_eos_eot_inf:
                logits_with_noise[:, :, 126081] = logits[:, :, 126348] = -torch.inf

            # 3. Remasking Strategy
            t_remask_start = time.perf_counter()
            if remasking == 'low_confidence':
                p = F.softmax(logits, dim=-1)
                x0_p = torch.squeeze(
                    torch.gather(p, dim=-1, index=torch.unsqueeze(x0, -1)), -1) # b, l
            elif remasking == 'random':
                x0_p = torch.rand((x0.shape[0], x0.shape[1]), device=x0.device)
            else:
                raise NotImplementedError(remasking)

            x0_p[:, prompt.shape[1] + (num_block + 1) * block_length:] = -np.inf

            x0 = torch.where(mask_index, x0, x)
            confidence = torch.where(mask_index, x0_p, -np.inf)

            transfer_index = torch.zeros_like(x0, dtype=torch.bool, device=x0.device)
            for j in range(confidence.shape[0]):
                _, select_index = torch.topk(confidence[j], k=num_transfer_tokens[j, i])
                transfer_index[j, select_index] = True
            x[transfer_index] = x0[transfer_index]

            if profiler is not None:
                save_token_state(
                    profiler.save_dir,
                    global_step,
                    prompt_index,
                    mask_index,
                    transfer_index,
                    num_block,
                    i,
                )
            
            synchronize()
            t_remask_end = time.perf_counter()
            
            step_end = time.perf_counter()
            step_duration = step_end - step_start
            
            step_details.append({
                "block_idx": num_block,
                "step_idx": i,
                "duration": step_duration,
                "forward_time": t_forward_end - t_forward_start,
                "sampling_time": t_sample_end - t_sample_start,
                "remasking_time": t_remask_end - t_remask_start,
                "num_masks": mask_index.sum().item()
            })

    synchronize()
    total_end = time.perf_counter()
    
    return x, step_details


def run_inference(model, tokenizer, prompt_text, steps=64, gen_length=64, block_length=32, profiler=None):
    """
    Runs inference and measures wall time with detailed profiling.
    """
    # Prepare input
    messages = [{"role": "user", "content": prompt_text}]
    formatted_prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, add_special_tokens=False)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Measure time
    start_time = time.perf_counter()
    
    with torch.no_grad():
        out, step_details = profiled_generate(
            model, 
            input_ids,
            attention_mask=attention_mask,
            steps=steps, 
            gen_length=gen_length, 
            block_length=block_length, 
            temperature=0., 
            cfg_scale=0., 
            remasking='low_confidence',
            profiler=profiler
        )
    
    end_time = time.perf_counter()
    wall_time = end_time - start_time
    
    # Decode output
    generated_text = tokenizer.batch_decode(out[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
    
    return generated_text, wall_time, step_details

In [ ]:
# Run Inference on a Subset
num_samples = 1
results = []
all_step_details = []

# Initialize Profiler
tensor_dir = '/root/autodl-tmp/profiling_results'
profiler = ActivationProfiler(model, target_layers=['ff_out'], save_dir=tensor_dir)
# profiler = ActivationProfiler(model, target_layers=['q_proj', 'k_proj', 'v_proj', 'attn_out'], save_dir=tensor_dir)

# Ensure hooks are cleared initially
profiler.clear()

print(f"Running inference on first {num_samples} samples...")

for i in range(num_samples):
    problem = ds[i]
    prompt = problem['prompt']
    task_id = problem['task_id']
    
    print(f"Processing {task_id}...")

	# Register hooks for this run
    profiler.register_hooks()

    try:
        # Using parameters from chat.py/generate.py examples
        output, duration, step_details = run_inference(
            model=model, 
            tokenizer=tokenizer, 
            prompt_text=prompt, 
            steps=128, 
            gen_length=128, 
            block_length=32,
            profiler=profiler
        )
        
        results.append({
            "task_id": task_id,
            "wall_time": duration,
            "output_length": len(output),
            "output": output
        })
        
		# Add task_id to each step detail
        for d in step_details:
            d['task_id'] = task_id
        all_step_details.extend(step_details)
        print(f"  Time: {duration:.4f}s")
    except Exception as e:
        print(f"  Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # Remove hooks to free memory and prevent crashes in subsequent runs
        profiler.clear()

profiler.clear() # Remove hooks after done

df_results = pd.DataFrame(results)
df_steps = pd.DataFrame(all_step_details)

## Statistics Analysis and Visualization

### Latency Analysis

In [ ]:
sns.set_style("whitegrid")

if not df_results.empty:
    # Wall-time stats
    stats = {
        "Mean Latency": df_results['wall_time'].mean(),
        "Median Latency": df_results['wall_time'].median(),
        "Std Dev": df_results['wall_time'].std(),
        "Min": df_results['wall_time'].min(),
        "Max": df_results['wall_time'].max(),
        "P95": df_results['wall_time'].quantile(0.95),
        "P99": df_results['wall_time'].quantile(0.99),
    }
    print("Wall Time Statistics (seconds):")
    for k, v in stats.items():
        print(f"{k}: {v:.4f}")

    plt.figure(figsize=(8, 4))
    sns.histplot(df_results['wall_time'], kde=True, bins=10)
    plt.title('Inference Wall Time Distribution')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Count')
    plt.axvline(df_results['wall_time'].mean(), color='r', linestyle='--', label=f"Mean: {df_results['wall_time'].mean():.2f}s")
    plt.legend()
    plt.show()

# Per-step breakdown
if not df_steps.empty:
    # Totals
    totals = df_steps[['forward_time', 'sampling_time', 'remasking_time']].sum()
    total_time = totals.sum()
    print("\nBreakdown (sum over all steps):")
    for name, val in totals.items():
        pct = 100.0 * val / total_time if total_time > 0 else 0.0
        print(f"  {name}: {val:.4f}s ({pct:.1f}%)")

    # Stacked bar per step
    plt.figure(figsize=(10, 4))
    steps_idx = np.arange(len(df_steps))
    fwd = df_steps['forward_time'].values
    smp = df_steps['sampling_time'].values
    rmk = df_steps['remasking_time'].values
    plt.bar(steps_idx, fwd, label='forward')
    plt.bar(steps_idx, smp, bottom=fwd, label='sampling')
    plt.bar(steps_idx, rmk, bottom=fwd + smp, label='remasking')
    plt.title('Per-step Latency Breakdown')
    plt.xlabel('Global step')
    plt.ylabel('Time (s)')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Optional: aggregate by block
    if 'block_idx' in df_steps.columns:
        block_agg = df_steps.groupby('block_idx')[['forward_time','sampling_time','remasking_time']].sum()
        block_agg.plot(kind='bar', stacked=True, figsize=(8,4), title='Latency Breakdown per Block')
        plt.xlabel('Block index')
        plt.ylabel('Time (s)')
        plt.tight_layout()
        plt.show()

### Token State Evolution

This section analyzes how token states evolve during the denoising process by examining the saved token state data.

#### Token State Analysis Functions

In [ ]:
# =========================
# Token state analysis utilities
# =========================


def load_token_state(base_dir, step):
    """Load token state data for a specific step."""
    step_dir = os.path.join(base_dir, f"step_{step}")
    token_state_path = os.path.join(step_dir, "token_state.pt")
    if os.path.exists(token_state_path):
        return torch.load(token_state_path, map_location="cpu")
    return None


def collect_token_evolution(base_dir, steps=None):
    """
    Collect token state evolution across all steps.
    Returns DataFrame with columns: step, token_idx, state, block_idx, inblock_step
    """
    if steps is None:
        step_dirs = sorted([d for d in os.listdir(base_dir) if d.startswith("step_")],
                          key=lambda x: int(x.split('_')[1]))
        steps = [int(d.split('_')[1]) for d in step_dirs]
    
    rows = []
    for step in steps:
        token_state = load_token_state(base_dir, step)
        if token_state is None:
            continue
            
        prompt_mask = token_state["prompt_mask"][0]  # [seq_len]
        mask_index = token_state["mask_index"][0]    # [seq_len]
        transfer_index = token_state["transfer_index"][0]  # [seq_len]
        block_idx = token_state["block_idx"]
        inblock_step = token_state["inblock_step"]
        
        seq_len = prompt_mask.shape[0]
        for token_idx in range(seq_len):
            # Determine token state
            if prompt_mask[token_idx]:
                state = "prompt"
            elif mask_index[token_idx]:
                if transfer_index[token_idx]:
                    state = "transferred"
                else:
                    state = "masked"
            else:
                state = "generated"
            
            rows.append({
                "step": step,
                "token_idx": token_idx,
                "state": state,
                "block_idx": block_idx,
                "inblock_step": inblock_step,
                "is_prompt": prompt_mask[token_idx].item(),
                "is_masked": mask_index[token_idx].item(),
                "is_transferred": transfer_index[token_idx].item()
            })
    
    return pd.DataFrame(rows)


def analyze_token_transitions(df_tokens):
    """Analyze how tokens transition between states."""
    transitions = []
    
    # Group by token and track state changes
    for token_idx in df_tokens['token_idx'].unique():
        token_history = df_tokens[df_tokens['token_idx'] == token_idx].sort_values('step')
        
        prev_state = None
        for _, row in token_history.iterrows():
            current_state = row['state']
            if prev_state is not None and prev_state != current_state:
                transitions.append({
                    'token_idx': token_idx,
                    'step': row['step'],
                    'from_state': prev_state,
                    'to_state': current_state,
                    'block_idx': row['block_idx']
                })
            prev_state = current_state
    
    return pd.DataFrame(transitions)


def get_token_coverage_stats(df_tokens):
    """Get statistics about token coverage at each step."""
    coverage_stats = []
    
    for step in sorted(df_tokens['step'].unique()):
        step_data = df_tokens[df_tokens['step'] == step]
        
        total_tokens = len(step_data)
        prompt_tokens = len(step_data[step_data['state'] == 'prompt'])
        masked_tokens = len(step_data[step_data['state'] == 'masked'])
        generated_tokens = len(step_data[step_data['state'] == 'generated'])
        transferred_tokens = len(step_data[step_data['state'] == 'transferred'])
        
        coverage_stats.append({
            'step': step,
            'total_tokens': total_tokens,
            'prompt_tokens': prompt_tokens,
            'masked_tokens': masked_tokens,
            'generated_tokens': generated_tokens,
            'transferred_tokens': transferred_tokens,
            'mask_ratio': masked_tokens / total_tokens if total_tokens > 0 else 0,
            'generated_ratio': generated_tokens / total_tokens if total_tokens > 0 else 0,
            'block_idx': step_data['block_idx'].iloc[0] if len(step_data) > 0 else None,
            'inblock_step': step_data['inblock_step'].iloc[0] if len(step_data) > 0 else None
        })
    
    return pd.DataFrame(coverage_stats)


#### Token Evolution Visualization

In [ ]:
# Collect token evolution data
print("Analyzing token state evolution...")
df_token_evolution = collect_token_evolution(tensor_dir)

if not df_token_evolution.empty:
    print(f"Loaded token states for {len(df_token_evolution['step'].unique())} steps")
    print(f"Sequence length: {df_token_evolution['token_idx'].max() + 1}")
    
    # Get coverage statistics
    df_coverage = get_token_coverage_stats(df_token_evolution)
    
    # 1. Token state heatmap over time
    plt.figure(figsize=(15, 8))
    
    # Create a matrix for visualization: steps x tokens
    steps = sorted(df_token_evolution['step'].unique())
    tokens = sorted(df_token_evolution['token_idx'].unique())
    
    # Map states to numbers for visualization
    state_map = {'prompt': 0, 'masked': 1, 'transferred': 2, 'generated': 3}
    state_matrix = np.full((len(steps), len(tokens)), -1)
    
    for i, step in enumerate(steps):
        step_data = df_token_evolution[df_token_evolution['step'] == step]
        for _, row in step_data.iterrows():
            j = row['token_idx']
            state_matrix[i, j] = state_map[row['state']]
    
    # Create custom colormap
    colors = ['blue', 'red', 'orange', 'green']  # prompt, masked, transferred, generated
    cmap = plt.matplotlib.colors.ListedColormap(colors)
    
    plt.imshow(state_matrix, cmap=cmap, aspect='auto', interpolation='nearest')
    plt.xlabel('Token Index')
    plt.ylabel('Denoising Step')
    plt.title('Token State Evolution During Denoising')
    
    # Add colorbar with labels
    cbar = plt.colorbar(ticks=[0, 1, 2, 3])
    cbar.set_ticklabels(['Prompt', 'Masked', 'Transferred', 'Generated'])
    
    plt.tight_layout()
    plt.show()
    
    # 2. Token coverage evolution
    plt.figure(figsize=(12, 6))
    
    plt.subplot(2, 2, 1)
    plt.plot(df_coverage['step'], df_coverage['mask_ratio'], 'r-', label='Masked Ratio')
    plt.plot(df_coverage['step'], df_coverage['generated_ratio'], 'g-', label='Generated Ratio')
    plt.xlabel('Step')
    plt.ylabel('Ratio')
    plt.title('Token Coverage Evolution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 2)
    plt.plot(df_coverage['step'], df_coverage['masked_tokens'], 'r-', label='Masked')
    plt.plot(df_coverage['step'], df_coverage['generated_tokens'], 'g-', label='Generated')
    plt.plot(df_coverage['step'], df_coverage['transferred_tokens'], 'orange', label='Transferred')
    plt.xlabel('Step')
    plt.ylabel('Token Count')
    plt.title('Token Count by State')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 3. Block-wise analysis
    plt.subplot(2, 2, 3)
    block_colors = plt.cm.Set3(np.linspace(0, 1, df_coverage['block_idx'].max() + 1))
    for block_idx in sorted(df_coverage['block_idx'].unique()):
        block_data = df_coverage[df_coverage['block_idx'] == block_idx]
        plt.plot(block_data['inblock_step'], block_data['mask_ratio'], 
                color=block_colors[block_idx], label=f'Block {block_idx}', marker='o')
    plt.xlabel('In-Block Step')
    plt.ylabel('Mask Ratio')
    plt.title('Mask Ratio by Block')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 4. Transfer rate analysis
    plt.subplot(2, 2, 4)
    transfer_rates = []
    for step in steps[1:]:  # Skip first step
        prev_step_data = df_token_evolution[df_token_evolution['step'] == step - 1]
        curr_step_data = df_token_evolution[df_token_evolution['step'] == step]
        
        prev_masked = len(prev_step_data[prev_step_data['state'] == 'masked'])
        curr_transferred = len(curr_step_data[curr_step_data['state'] == 'transferred'])
        
        transfer_rate = curr_transferred / prev_masked if prev_masked > 0 else 0
        transfer_rates.append(transfer_rate)
    
    plt.plot(steps[1:], transfer_rates, 'b-o')
    plt.xlabel('Step')
    plt.ylabel('Transfer Rate')
    plt.title('Token Transfer Rate per Step')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nToken Evolution Summary:")
    print(f"Total denoising steps: {len(steps)}")
    print(f"Initial masked tokens: {df_coverage.iloc[0]['masked_tokens']}")
    print(f"Final generated tokens: {df_coverage.iloc[-1]['generated_tokens']}")
    print(f"Average tokens transferred per step: {df_coverage['transferred_tokens'].mean():.2f}")
    
else:
    print("No token state data found. Make sure profiling was run with token state saving enabled.")

#### Detailed Token Transition Analysis

In [ ]:
if not df_token_evolution.empty:
    # Analyze token transitions
    df_transitions = analyze_token_transitions(df_token_evolution)
    
    if not df_transitions.empty:
        print(f"\nFound {len(df_transitions)} token state transitions")
        
        # Transition frequency analysis
        transition_counts = df_transitions.groupby(['from_state', 'to_state']).size().reset_index(name='count')
        
        plt.figure(figsize=(10, 6))
        
        # Create transition matrix for heatmap
        states = ['prompt', 'masked', 'transferred', 'generated']
        transition_matrix = pd.pivot_table(transition_counts, 
                                         index='from_state', 
                                         columns='to_state', 
                                         values='count', 
                                         fill_value=0)
        
        # Reindex to ensure all states are included
        transition_matrix = transition_matrix.reindex(index=states, columns=states, fill_value=0)
        
        sns.heatmap(transition_matrix, annot=True, cmap='Blues')
        plt.title('Token State Transition Matrix')
        plt.xlabel('To State')
        plt.ylabel('From State')
        plt.tight_layout()
        plt.show()
        
        # Transition timing analysis
        plt.figure(figsize=(12, 4))
        
        for transition_type in transition_counts[['from_state', 'to_state']].apply(
            lambda x: f"{x['from_state']} → {x['to_state']}", axis=1).unique():
            
            from_state, to_state = transition_type.split(' → ')
            transition_data = df_transitions[
                (df_transitions['from_state'] == from_state) & 
                (df_transitions['to_state'] == to_state)
            ]
            
            if len(transition_data) > 0:
                step_counts = transition_data['step'].value_counts().sort_index()
                plt.plot(step_counts.index, step_counts.values, 
                        marker='o', label=transition_type, alpha=0.7)
        
        plt.xlabel('Step')
        plt.ylabel('Number of Transitions')
        plt.title('Token State Transitions Over Time')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print("\nTransition Summary:")
        print(transition_counts.to_string(index=False))
    else:
        print("No state transitions detected.")


#### Token-Level Detailed Analysis


In [ ]:
if not df_token_evolution.empty:
    # Analyze specific token ranges
    seq_len = df_token_evolution['token_idx'].max() + 1
    prompt_len = df_token_evolution[df_token_evolution['state'] == 'prompt']['token_idx'].max() + 1
    gen_len = seq_len - prompt_len
    
    print(f"\nSequence Analysis:")
    print(f"Total sequence length: {seq_len}")
    print(f"Prompt length: {prompt_len}")
    print(f"Generation length: {gen_len}")
    
    # Focus on generation region
    gen_tokens = df_token_evolution[df_token_evolution['token_idx'] >= prompt_len].copy()
    
    if not gen_tokens.empty:
        plt.figure(figsize=(15, 8))
        
        # Generation region heatmap
        gen_steps = sorted(gen_tokens['step'].unique())
        gen_token_indices = sorted(gen_tokens['token_idx'].unique())
        
        state_matrix_gen = np.full((len(gen_steps), len(gen_token_indices)), -1)
        
        for i, step in enumerate(gen_steps):
            step_data = gen_tokens[gen_tokens['step'] == step]
            for _, row in step_data.iterrows():
                j = gen_token_indices.index(row['token_idx'])
                state_matrix_gen[i, j] = state_map[row['state']]
        
        plt.imshow(state_matrix_gen, cmap=cmap, aspect='auto', interpolation='nearest')
        plt.xlabel('Generation Token Index')
        plt.ylabel('Denoising Step')
        plt.title('Token State Evolution in Generation Region')
        
        # Adjust x-axis labels to show actual token indices
        n_ticks = min(10, len(gen_token_indices))
        tick_indices = np.linspace(0, len(gen_token_indices)-1, n_ticks, dtype=int)
        plt.xticks(tick_indices, [gen_token_indices[i] for i in tick_indices])
        
        cbar = plt.colorbar(ticks=[0, 1, 2, 3])
        cbar.set_ticklabels(['Prompt', 'Masked', 'Transferred', 'Generated'])
        
        plt.tight_layout()
        plt.show()
        
        # Block boundary analysis
        block_transitions = gen_tokens.groupby(['step', 'block_idx']).size().reset_index(name='count')
        
        plt.figure(figsize=(12, 4))
        for block_idx in sorted(block_transitions['block_idx'].unique()):
            block_data = df_coverage[df_coverage['block_idx'] == block_idx]
            plt.axvspan(block_data['step'].min(), block_data['step'].max(), 
                       alpha=0.3, label=f'Block {block_idx}')
        
        plt.plot(df_coverage['step'], df_coverage['generated_tokens'], 'g-', linewidth=2)
        plt.xlabel('Step')
        plt.ylabel('Generated Tokens')
        plt.title('Token Generation Progress with Block Boundaries')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

print("Token state evolution analysis complete!")

### Tensor Distribution

#### Function Definitions

In [ ]:
# =========================
# Tensor loader utilities (saved to disk during inference)
# =========================
import glob, re

_layer_re = re.compile(
    r".*_(?:layers|transformer_blocks)_(\d+).*_(q_proj|k_proj|v_proj|attn_out|ff_proj|up_proj|ff_out)\.pt$"
)
tensor_dir = '/root/autodl-tmp/profiling_results'


def load_step_tensors(base_dir, step, selector_substrings):
    """
    Returns {layer_file: tensor} for files containing any selector substring.
    selector_substrings: list[str], e.g., ['q_proj', 'k_proj'].
    """
    step_dir = os.path.join(base_dir, f"step_{step}")
    tensors = {}
    if not os.path.isdir(step_dir):
        return tensors
    for f in glob.glob(os.path.join(step_dir, "*.pt")):
        base = os.path.basename(f)
        if any(s in base for s in selector_substrings):
            tensors[base] = torch.load(f, map_location="cpu")
    return tensors


def parse_layer_and_kind(layer_file_basename):
    m = _layer_re.match(layer_file_basename)
    if m:
        return int(m.group(1)), m.group(2)
    for k in ['q_proj','k_proj','v_proj','attn_out','ff_proj','up_proj','ff_out']:
        if k in layer_file_basename:
            return None, k
    return None, None


def reduce_stat(t, how="mean_abs", sample=None):
    """
    Reduce a tensor [batch, seq, hidden] to a scalar.
    how: 'mean_abs' | 'max_abs' | 'std' | 'l2'
    sample: dict to subsample dims, e.g., {'seq': 256, 'hidden': 512}
    """
    x = t
    # optional subsample to reduce memory/plot time
    if sample:
        b, s, h = x.shape
        if 'seq' in sample:
            x = x[:, torch.linspace(0, s-1, steps=min(sample['seq'], s), dtype=torch.long), :]
        if 'hidden' in sample:
            x = x[:, :, torch.linspace(0, h-1, steps=min(sample['hidden'], h), dtype=torch.long)]
    x = x.float()
    if how == "mean_abs":
        return x.abs().mean().item()
    if how == "max_abs":
        return x.abs().amax().item()
    if how == "std":
        return x.std().item()
    if how == "l2":
        return x.pow(2).mean().sqrt().item()
    raise ValueError(how)


def collect_series(base_dir, selectors=('q_proj','k_proj'), how="mean_abs", sample=None, steps=None):
    """
    Scan steps->layers and compute a scalar stat per (step, layer, kind).
    Returns DataFrame columns: step, layer_idx, kind, value
    """
    rows = []
    # infer steps if not provided
    if steps is None:
        step_dirs = sorted([d for d in os.listdir(base_dir) if d.startswith("step_")],
                           key=lambda x: int(x.split('_')[1]))
        steps = [int(d.split('_')[1]) for d in step_dirs]
    for step in steps:
        tensors = load_step_tensors(base_dir, step, selectors)
        for name, t in tensors.items():
            layer_idx, kind = parse_layer_and_kind(name)
            if kind is None: 
                continue
            val = reduce_stat(t, how=how, sample=sample)
            rows.append({"step": step, "layer_idx": layer_idx, "kind": kind, "value": val, "name": name})
    return pd.DataFrame(rows)


def flatten_activation(t, mode="batch_seq_mean"):
    x = t.float()
    if mode == "batch_seq_mean":
        return x.mean(dim=(0, 1))                        # [hidden]
    if mode == "batch_mean":
        return x.mean(dim=0).reshape(-1, x.size(-1))     # [seq, hidden]
    if mode == "none":
        return x.reshape(-1, x.size(-1))                 # [tokens, hidden]
    raise ValueError(mode)


def _token_matrix(x):
    x = x.float()
    if x.ndim == 2:
        return x
    if x.ndim == 3:
        return x.view(-1, x.size(-1))
    raise ValueError(f"Unsupported tensor shape {tuple(x.shape)} for token-wise mode")


def cosine_similarity_tensors(t_a, t_b, mode="batch_seq_mean"):
    if mode == "token-wise":
        a = _token_matrix(t_a)
        b = _token_matrix(t_b)
        sims = F.cosine_similarity(a, b, dim=-1)
        return sims.cpu().numpy()
    elif mode == 'batch_seq_mean':
        a = flatten_activation(t_a, 'batch_seq_mean')
        b = flatten_activation(t_b, 'batch_seq_mean')
        if a.ndim == 1:
            return F.cosine_similarity(a.unsqueeze(0), b.unsqueeze(0)).item()
        sims = F.cosine_similarity(a, b, dim=-1)
        return sims.mean().item()
    elif mode == 'batch_mean':
        a = flatten_activation(t_a, 'batch_mean')          # [seq, hidden]
        b = flatten_activation(t_b, 'batch_mean')
        sims = F.cosine_similarity(a, b, dim=-1)
        return sims.mean().item()
    else:
        raise ValueError(mode)


def expand_tokenwise_similarity(df_sim):
    rows = []
    for _, row in df_sim.iterrows():
        sims = row["similarity"]
        if isinstance(sims, np.ndarray) and row["layer_idx"] is not None:
            for token_idx, val in enumerate(sims):
                rows.append({
                    "step_pair": f"{row['step_a']}→{row['step_b']}",
                    "layer_idx": row["layer_idx"],
                    "name": row["name"],
                    "token_idx": token_idx,
                    "token_similarity": float(val),
                })
    return pd.DataFrame(rows)


def collect_similarity_for_pairs(base_dir, step_pairs, selector="attn_out", tensor_names=None, mode="batch_seq_mean"):
    rows = []
    for step_a, step_b in step_pairs:
        tensors_a = load_step_tensors(base_dir, step_a, [selector])
        tensors_b = load_step_tensors(base_dir, step_b, [selector])
        candidates = tensors_a.keys() if not tensor_names else [
            name for name in tensors_a if any(tag in name for tag in tensor_names)
        ]
        for name in candidates:
            if name not in tensors_b:
                continue
            layer_idx, kind = parse_layer_and_kind(name)
            sim = cosine_similarity_tensors(tensors_a[name], tensors_b[name], mode=mode)
            rows.append({
                "step_a": step_a,
                "step_b": step_b,
                "layer_idx": layer_idx,
                "kind": kind,
                "name": name,
                "similarity": sim,
            })
    return pd.DataFrame(rows)

#### Run Analysis

In [ ]:
# Heatmap of mean_abs over (steps x layers) for q_proj
df_q = collect_series(tensor_dir, selectors=('q_proj',), how="mean_abs", sample={"seq":256, "hidden":512})
if not df_q.empty and df_q['layer_idx'].notna().all():
    pivot = df_q.pivot_table(index="step", columns="layer_idx", values="value", aggfunc="mean").sort_index()
    plt.figure(figsize=(12, 5))
    sns.heatmap(pivot.T, cmap="viridis", cbar_kws={"label": "mean|x|"}, robust=True)
    plt.title("q_proj mean_abs across layers (cols) and steps (rows)")
    plt.xlabel("Step")
    plt.ylabel("Layer index")
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare q_proj vs k_proj averaged over layers per step
df_qk = collect_series(tensor_dir, selectors=('q_proj','k_proj'), how="max_abs", sample={"seq":256, "hidden":512})
if not df_qk.empty:
    df_avg = df_qk.groupby(['step','kind'])['value'].mean().reset_index()
    plt.figure(figsize=(10,4))
    sns.lineplot(data=df_avg, x='step', y='value', hue='kind')
    plt.title("q_proj vs k_proj (avg over layers) — max_abs")
    plt.xlabel("Step")
    plt.ylabel("max|x|")
    plt.tight_layout()
    plt.show()

In [ ]:
# Distribution at a given step for specific layers/kind
TARGET_STEP = df_q['step'].max() if not df_q.empty else None
if TARGET_STEP is not None:
    tensors = load_step_tensors(tensor_dir, TARGET_STEP, ['q_proj'])
    # pick first layer entry for demo
    if tensors:
        name, tensor = sorted(tensors.items())[0]
        x = tensor.float().view(-1).numpy()
        plt.figure(figsize=(8,4))
        sns.histplot(x, bins=100, kde=True)
        plt.title(f"Distribution of {name} at step={TARGET_STEP}")
        plt.xlabel("Activation value")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.show()

In [ ]:
# Enhanced cosine similarity analysis with comprehensive token state tracking
def enhanced_cosine_similarity_with_full_evolution():
    """
    Analyze cosine similarity with complete token state evolution tracking.
    First loads all token states to understand the full evolution.
    """
    STEP_PAIRS = [(0, 1), (31, 32), (63, 64), (126, 127)]
    TENSOR_NAMES = None
    MODE = "token-wise"
    
    # First, load ALL available token states to track evolution
    def load_all_token_states(base_dir):
        """Load token states for all available steps."""
        token_states = {}
        step_dirs = sorted([d for d in os.listdir(base_dir) if d.startswith("step_")],
                          key=lambda x: int(x.split('_')[1]))
        
        for step_dir in step_dirs:
            step_num = int(step_dir.split('_')[1])
            token_state_path = os.path.join(base_dir, step_dir, "token_state.pt")
            if os.path.exists(token_state_path):
                token_states[step_num] = torch.load(token_state_path, map_location="cpu")
        
        return token_states
    
    def build_comprehensive_token_evolution(all_token_states):
        """
        Build comprehensive token evolution by tracking each token across all steps.
        """
        if not all_token_states:
            return None
            
        # Get the sequence length from the first available state
        first_step = min(all_token_states.keys())
        seq_len = len(all_token_states[first_step]["prompt_mask"][0])
        
        # Initialize token evolution tracking
        token_evolution = {}  # token_idx -> list of states across steps
        
        for token_idx in range(seq_len):
            token_evolution[token_idx] = []
        
        # Track evolution across all steps
        sorted_steps = sorted(all_token_states.keys())
        
        for step in sorted_steps:
            state = all_token_states[step]
            prompt_mask = state["prompt_mask"][0]
            mask_index = state["mask_index"][0] 
            transfer_index = state["transfer_index"][0]
            
            for token_idx in range(seq_len):
                if prompt_mask[token_idx]:
                    token_state = "prompt"
                elif mask_index[token_idx]:
                    if transfer_index[token_idx]:
                        token_state = "transferred"
                    else:
                        token_state = "masked"
                else:
                    # Not prompt, not currently masked = generated
                    token_state = "generated"
                
                token_evolution[token_idx].append({
                    'step': step,
                    'state': token_state,
                    'block_idx': state["block_idx"],
                    'inblock_step': state["inblock_step"]
                })
        
        return token_evolution, sorted_steps
    
    def get_token_state_at_step(token_evolution, step):
        """Get token states for all tokens at a specific step."""
        token_states = []
        for token_idx in range(len(token_evolution)):
            # Find the state at this step
            for state_info in token_evolution[token_idx]:
                if state_info['step'] == step:
                    token_states.append(state_info['state'])
                    break
            else:
                # If step not found, assume it's the last known state
                if token_evolution[token_idx]:
                    token_states.append(token_evolution[token_idx][-1]['state'])
                else:
                    token_states.append("unknown")
        return token_states
    
    # Load all token states
    print("Loading all token states for comprehensive evolution tracking...")
    all_token_states = load_all_token_states(tensor_dir)
    
    if not all_token_states:
        print("No token state data found.")
        return
    
    print(f"Loaded token states for {len(all_token_states)} steps: {sorted(all_token_states.keys())}")
    
    # Build comprehensive evolution
    token_evolution, all_steps = build_comprehensive_token_evolution(all_token_states)
    
    if token_evolution is None:
        print("Could not build token evolution.")
        return
    
    # Get sequence information
    first_step = min(all_token_states.keys())
    prompt_len = all_token_states[first_step]["prompt_mask"][0].sum().item()
    seq_len = len(token_evolution)
    gen_len = seq_len - prompt_len
    block_length = 32  # From your configuration
    
    # Calculate block boundaries
    block_boundaries = []
    for i in range(0, gen_len, block_length):
        block_boundaries.append(prompt_len + i)
    if prompt_len + gen_len not in block_boundaries:
        block_boundaries.append(prompt_len + gen_len)
    
    print(f"Sequence info: Total={seq_len}, Prompt={prompt_len}, Generation={gen_len}")
    print(f"Block boundaries: {block_boundaries}")
    
    # Collect similarity data
    df_sim = collect_similarity_for_pairs(
        tensor_dir,
        STEP_PAIRS,
        selector="attn_out",
        tensor_names=TENSOR_NAMES,
        mode=MODE,
    )
    
    if df_sim.empty:
        print("No overlapping tensors for the requested pairs.")
        return
    
    if MODE == "token-wise":
        df_tok = expand_tokenwise_similarity(df_sim)
        if df_tok.empty:
            print("Token-wise similarities lack valid layer indices to plot.")
            return
            
        for step_pair, group in df_tok.groupby("step_pair"):
            step_a, step_b = map(int, step_pair.split('→'))
            
            # Get token states using comprehensive evolution
            token_states_a = get_token_state_at_step(token_evolution, step_a)
            token_states_b = get_token_state_at_step(token_evolution, step_b)
            
            # Get block info
            block_idx_a = all_token_states[step_a]["block_idx"]
            block_idx_b = all_token_states[step_b]["block_idx"]
            
            # Create the similarity pivot table
            pivot = (
                group.pivot_table(index="layer_idx", columns="token_idx", values="token_similarity", aggfunc="mean")
                .sort_index()
            )
            
            # Create enhanced visualization
            fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))
            
            # Main heatmap
            im = ax1.imshow(
                pivot.values,
                cmap="coolwarm",
                vmin=0,
                vmax=1,
                aspect='auto',
                interpolation='nearest'
            )
            
            # Add block boundary lines
            for boundary in block_boundaries:
                if boundary < seq_len:
                    ax1.axvline(x=boundary-0.5, color='black', linestyle='-', linewidth=2, alpha=0.7)
            
            # Add prompt boundary
            ax1.axvline(x=prompt_len-0.5, color='purple', linestyle='-', linewidth=3, alpha=0.8)
            
            ax1.set_title(f"{step_pair}: Token-wise Cosine Similarity (attn_out)\nBlack=Block boundaries, Purple=Prompt boundary")
            ax1.set_xlabel("Token Index")
            ax1.set_ylabel("Layer Index")
            
            # Set y-axis ticks to show layer indices
            if len(pivot.index) <= 20:
                ax1.set_yticks(range(len(pivot.index)))
                ax1.set_yticklabels(pivot.index)
            
            # Add colorbar
            plt.colorbar(im, ax=ax1, label="Cosine Similarity")
            
            # Token state visualization for step A
            state_colors = {'prompt': 0, 'masked': 1, 'transferred': 2, 'generated': 3, 'unknown': 4}
            state_array_a = [state_colors.get(state, 4) for state in token_states_a]
            
            ax2.imshow([state_array_a], cmap=plt.cm.Set3, aspect='auto', interpolation='nearest')
            ax2.set_title(f"Token States at Step {step_a} (Block {block_idx_a})")
            ax2.set_xlabel("Token Index")
            ax2.set_yticks([])
            
            # Add block boundaries
            for boundary in block_boundaries:
                if boundary < seq_len:
                    ax2.axvline(x=boundary-0.5, color='black', linestyle='-', linewidth=2, alpha=0.7)
            ax2.axvline(x=prompt_len-0.5, color='purple', linestyle='-', linewidth=3, alpha=0.8)
            
            # Token state visualization for step B
            state_array_b = [state_colors.get(state, 4) for state in token_states_b]
            
            ax3.imshow([state_array_b], cmap=plt.cm.Set3, aspect='auto', interpolation='nearest')
            ax3.set_title(f"Token States at Step {step_b} (Block {block_idx_b})")
            ax3.set_xlabel("Token Index")
            ax3.set_yticks([])
            
            # Add block boundaries
            for boundary in block_boundaries:
                if boundary < seq_len:
                    ax3.axvline(x=boundary-0.5, color='black', linestyle='-', linewidth=2, alpha=0.7)
            ax3.axvline(x=prompt_len-0.5, color='purple', linestyle='-', linewidth=3, alpha=0.8)
            
            # Add legend for token states
            from matplotlib.patches import Patch
            legend_elements = [
                Patch(facecolor=plt.cm.Set3(0), label='Prompt'),
                Patch(facecolor=plt.cm.Set3(1), label='Masked'),
                Patch(facecolor=plt.cm.Set3(2), label='Transferred'),
                Patch(facecolor=plt.cm.Set3(3), label='Generated')
            ]
            ax3.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))
            
            plt.tight_layout()
            plt.show()
            
            # Print summary statistics with corrected counts
            print(f"\nStep Pair {step_pair} Summary:")
            print(f"  Step {step_a}: Block {block_idx_a}")
            print(f"    Prompt tokens: {sum(1 for s in token_states_a if s == 'prompt')}")
            print(f"    Masked tokens: {sum(1 for s in token_states_a if s == 'masked')}")
            print(f"    Generated tokens: {sum(1 for s in token_states_a if s == 'generated')}")
            print(f"    Transferred tokens: {sum(1 for s in token_states_a if s == 'transferred')}")
            
            print(f"  Step {step_b}: Block {block_idx_b}")
            print(f"    Prompt tokens: {sum(1 for s in token_states_b if s == 'prompt')}")
            print(f"    Masked tokens: {sum(1 for s in token_states_b if s == 'masked')}")
            print(f"    Generated tokens: {sum(1 for s in token_states_b if s == 'generated')}")
            print(f"    Transferred tokens: {sum(1 for s in token_states_b if s == 'transferred')}")
            
            # Analyze similarity by token type
            token_type_similarities = {}
            for token_type in ['prompt', 'masked', 'generated', 'transferred']:
                type_indices_a = [i for i, state in enumerate(token_states_a) if state == token_type]
                type_indices_b = [i for i, state in enumerate(token_states_b) if state == token_type]
                common_indices = set(type_indices_a) & set(type_indices_b)
                
                if common_indices:
                    # Get similarity values for this token type
                    type_similarities = []
                    for token_idx in common_indices:
                        if token_idx in group['token_idx'].values:
                            token_sim = group[group['token_idx'] == token_idx]['token_similarity'].mean()
                            type_similarities.append(token_sim)
                    
                    if type_similarities:
                        token_type_similarities[token_type] = np.mean(type_similarities)
            
            print(f"  Average similarity by token type:")
            for token_type, avg_sim in token_type_similarities.items():
                print(f"    {token_type}: {avg_sim:.4f}")
            print()

# Run the enhanced analysis with comprehensive token evolution
enhanced_cosine_similarity_with_full_evolution()